In [1]:
import os
import pandas as pd

def read_and_structure_csv_files(folder_path, exclude_list):
    # Initialize an empty list to store DataFrames
    data_frames = []
    file_name_list = []

    # Iterate through all files in the folder
    for file_name in os.listdir(folder_path):
        # Check if the file is a CSV
        # if file_name == 'Base Model.csv' or 'Default' in file_name or 'Conn' in file_name:
        #exclude_conditions = ['Base Model.csv', 'Conn']
        
        if any(cond in file_name for cond in exclude_list):
            continue
        if file_name.endswith('.csv'):
            print(file_name)
            file_name_list.append(file_name)
            file_path = os.path.join(folder_path, file_name)
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)
            df.rename(columns={'True': 'Tricked'}, inplace=True)
            df['Tricked'] = df['Tricked'].astype(str)
            df['Tricked'] = df['Tricked'].str.strip()
            df['Identifier'] = df['Identifier'].replace('Unknown', 'BPDA')
            # Append the DataFrame to the list
            df['filename'] = file_name
            data_frames.append(df)
            

    # Concatenate all DataFrames into a single DataFrame
    #combined_df = pd.concat(data_frames, ignore_index=True)

    return data_frames, file_name_list

In [2]:
# Specify the folder path containing the CSV files
mod = "lenet_soft"
folder_path = f'csv_{mod}/'
# exclude_list = ['Base Model.csv', 'Conn']
exclude_list = ['Base Model.csv', 'Conn']

# Call the function to read and structure CSV files
structured_data, file_name_list = read_and_structure_csv_files(folder_path, exclude_list)

SCP Default.csv
Unstruct Adv.csv
SCP Adv.csv
Trivial Adv.csv
Unstruct Default.csv
Trivial Default.csv


In [3]:
base_model_df = pd.read_csv(f'{folder_path}Base Model.csv')
base_model_df.rename(columns={'True': 'Tricked'}, inplace=True)
base_model_df['Identifier'] = base_model_df['Identifier'].replace('Unknown', 'BPDA')
base_model_df['filename'] = "Base Model.csv"
base_model_df['Tricked'] = base_model_df['Tricked'].astype(str)
base_model_df.iloc[:, 2] = base_model_df.iloc[:, 2].str.strip()

In [4]:
base_model_filtered = base_model_df[
    (base_model_df.iloc[:, 1] != 'Unknown') &
    (base_model_df.iloc[:, 2] == "True") &
    (base_model_df.iloc[:, 3] >= 0)
    ]

print(base_model_filtered)


    Identifier  Evaluation Tricked  2-Norm for Adv.-Image  Class  \
28    DeepFool        2358    True               0.757166      1   
147        PGD        9638    True               0.606185      9   

           filename  
28   Base Model.csv  
147  Base Model.csv  


In [5]:
combined_other_filtered = [data[
                               (data.iloc[:, 0] != 'Unknown') &
                               (data.iloc[:, 2] == "False") &
                               (data.iloc[:, 3] > 0)
                               ] for data in structured_data]

print(combined_other_filtered[0])



    Identifier  Evaluation Tricked  2-Norm for Adv.-Image  Class  \
0     DeepFool        2772   False               0.132019      5   
2     DeepFool        5980   False               0.346427      0   
3     DeepFool        1662   False               1.038896      6   
4     DeepFool         779   False               0.376378      5   
11    DeepFool        8653   False               0.067752      5   
..         ...         ...     ...                    ...    ...   
395   OnePixel        1867   False               0.917267      1   
396   OnePixel        6026   False               0.922992      7   
397   OnePixel        5492   False               0.849129      3   
398   OnePixel        4782   False               0.791578      3   
399   OnePixel        1693   False               0.988667      5   

            filename  
0    SCP Default.csv  
2    SCP Default.csv  
3    SCP Default.csv  
4    SCP Default.csv  
11   SCP Default.csv  
..               ...  
395  SCP Default.csv  

In [6]:
def count_matches(row):
    return sum(
        any(
            (other_df.iloc[:, 0] == row.iloc[0]) &
            (other_df.iloc[:, 1] == row.iloc[1])
        ) for other_df in combined_other_filtered
    )

base_model_filtered['match_count'] = base_model_filtered.apply(count_matches, axis=1)
base_model_filtered

/var/folders/s7/fxyhphlx22x1_wch4sxjtr1w0000gn/T/ipykernel_58562/905639066.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_model_filtered['match_count'] = base_model_filtered.apply(count_matches, axis=1)


,Identifier,Evaluation,Tricked,2-Norm for Adv.-Image,Class,filename,match_count
28,DeepFool,2358,True,0.757166,1,Base Model.csv,4
147,PGD,9638,True,0.606185,9,Base Model.csv,2


In [7]:
max_matches = base_model_filtered['match_count'].max()
most_matched_rows = base_model_filtered[base_model_filtered['match_count'] == max_matches]


most_matched_rows

,Identifier,Evaluation,Tricked,2-Norm for Adv.-Image,Class,filename,match_count
28,DeepFool,2358,True,0.757166,1,Base Model.csv,4


In [8]:
all_matched_compare = pd.DataFrame()
# for df in combined_other_filtered:
#     matched = df[
#         df['Evaluation'].isin(most_matched_rows['Evaluation']) &
#         df['Identifier'].isin(most_matched_rows['Identifier'])
#         ]
#     all_matched_compare = pd.concat([all_matched_compare, matched])
# for df in combined_other_filtered:
#     matched = df[
#         (df['Evaluation'] == most_matched_rows['Evaluation']) &
#         (df['Identifier'] == most_matched_rows['Identifier'])
#         ] # Select only the required columns
#     all_matched_compare = pd.concat([all_matched_compare, matched], ignore_index=True)
    
# all_matched_compare = pd.dataframe(
#     any(
#         (other_df['Evaluation'] == most_matched_rows['Identifier']) &
#         (other_df['Identifier'] == most_matched_rows['Evaluation'])
#     ) for other_df in combined_other_filtered
# )

matched_dfs = []

# Iterate through each DataFrame in the list
for other_df in combined_other_filtered:
    # Filter rows where 'Evaluation' and 'Identifier' match with 'most_matched_rows'
    matched_rows = other_df[
        other_df.apply(lambda row: any(
            (row['Evaluation'] == most_row['Evaluation'] and row['Identifier'] == most_row['Identifier'])
            for _, most_row in most_matched_rows.iterrows()
        ), axis=1)
    ]
    # Append the filtered DataFrame to the list
    matched_dfs.append(matched_rows)
matched_dfs.append(most_matched_rows)

# Concatenate all matched DataFrames into one
all_matched_compare = pd.concat(matched_dfs, ignore_index=True)
all_matched_compare.sort_values(by=['Identifier', 'Evaluation'], ignore_index=True)

all_matched_compare.sort_values(by=['Identifier', 'Evaluation'], ignore_index=True).to_csv(f"{mod}_base_neg.csv", index=False, sep=';', decimal=',')

all_matched_compare.sort_values(by=['Identifier', 'Evaluation'], ignore_index=True)
    
#all_matched_compare['filename'] = file_name_list
#most_matched_rows['filename'] = "Base Model.csv"
# all_matched_compare = pd.concat([all_matched_compare, most_matched_rows], ignore_index=True)
# all_matched_compare.sort_values(by=['Identifier', 'Evaluation'], ignore_index=True)

,Identifier,Evaluation,Tricked,2-Norm for Adv.-Image,Class,filename,match_count
0,DeepFool,2358,False,0.936218,1.0,SCP Default.csv,NaN
1,DeepFool,2358,False,0.816113,1.0,Unstruct Adv.csv,NaN
2,DeepFool,2358,False,1.154948,1.0,Trivial Adv.csv,NaN
3,DeepFool,2358,False,1.250886,1.0,Unstruct Default.csv,NaN
4,DeepFool,2358,True,0.757166,1.0,Base Model.csv,4.0


In [9]:
# Function to determine the name with the lowest value in each group
def find_lowest_name(group):
    return group.loc[group['2-Norm for Adv.-Image'].idxmin(), 'filename']

# Group by 'Identifier' and 'Evaluation' and apply the function to find the lowest name
lowest_names = all_matched_compare.groupby(['Identifier', 'Evaluation']).apply(find_lowest_name).reset_index(name='LowestName')

print(lowest_names)
# Count the occurrences of each name being the lowest
name_counts = lowest_names['LowestName'].value_counts().reset_index()
name_counts.columns = ['filename', 'Count']

export_min = "minValAdv/"
export_base_fail = "baseFail/"

#name_counts.to_csv(f"{export_min}res_ultraHard_minAdv_ohneUnstr_ohneAdv.csv", index=False)
#all_matched_compare.sort_values(by=['Identifier', 'Evaluation'], ignore_index=True).to_csv(f"{export_base_fail}res_ultraHard_ohneUnstr_ohneAdv.csv", index=False)
#all_matched_compare.sort_values(by=['Identifier', 'Evaluation'], ignore_index=True).to_csv(f"{export_base_fail}lenet_ultraHard_ohneUnstr_ohneAdv.csv", index=False)
#all_matched_compare.to_csv(f"{export_min}lenet_all_failed_ohneUnstruct_ohneAdv.csv", index=False)


print(all_matched_compare)
print(name_counts)

  Identifier  Evaluation      LowestName
0   DeepFool        2358  Base Model.csv
  Identifier  Evaluation Tricked  2-Norm for Adv.-Image  Class  \
0   DeepFool        2358   False               0.936218    1.0   
1   DeepFool        2358   False               0.816113    1.0   
2   DeepFool        2358   False               1.154948    1.0   
3   DeepFool        2358   False               1.250886    1.0   
4   DeepFool        2358    True               0.757166    1.0   

               filename  match_count  
0       SCP Default.csv          NaN  
1      Unstruct Adv.csv          NaN  
2       Trivial Adv.csv          NaN  
3  Unstruct Default.csv          NaN  
4        Base Model.csv          4.0  
         filename  Count
0  Base Model.csv      1
